# Pandas basics

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../../DataScience/datasets/titanic/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Crosstabs --> pd.crosstab()

In [3]:
# Compute a simple cross-tabulation of 2 factors.
pd.crosstab(df['Pclass'], df['Survived'], margins=True)

Survived,0,1,All
Pclass,,,
1,80,136,216
2,97,87,184
3,372,119,491
All,549,342,891


In [4]:
# Compute a cross-tabulation of 3 factors.
pd.crosstab([df['Sex'], df['Pclass']], df['Survived'], margins=True)

Survived         0    1  All
Sex    Pclass               
female 1         3   91   94
       2         6   70   76
       3        72   72  144
male   1        77   45  122
       2        91   17  108
       3       300   47  347
All            549  342  891

## Binning values into categories --> pd.cut()

In [5]:
df_ages = df.loc[:, ['PassengerId','Age']]
df_ages.head(10)

,PassengerId,Age
0,1,22.0
1,2,38.0
2,3,26.0
3,4,35.0
4,5,35.0
5,6,NaN
6,7,54.0
7,8,2.0
8,9,27.0
9,10,14.0


In [6]:
# Define the bin limits
bin_limits = [0, 5, 10, 25, 50, 100]
# Define the corresponding bin names
bin_names = ['Infant', 'Child', 'Teenager', 'Adult', 'Elderly']

# Bin the age column
df_ages['Category'] = pd.cut(df_ages['Age'], bins=bin_limits, labels=bin_names)
df_ages.head(10)

,PassengerId,Age,Category
0,1,22.0,Teenager
1,2,38.0,Adult
2,3,26.0,Adult
3,4,35.0,Adult
4,5,35.0,Adult
5,6,NaN,NaN
6,7,54.0,Elderly
7,8,2.0,Infant
8,9,27.0,Adult
9,10,14.0,Teenager


## Pivot table --> pd.pivot_table()

In [7]:
# Count number of missing values from data frame for each column
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
# Retrieve the 10 first rows where Age is missing
df_missing_age = df[df['Age'].isnull()].head(10)
df_missing_age

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q
29,30,0,3,"Todoroff, Mr. Lalio",male,NaN,0,0,349216,7.8958,NaN,S
31,32,1,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,NaN,1,0,PC 17569,146.5208,B78,C
32,33,1,3,"Glynn, Miss. Mary Agatha",female,NaN,0,0,335677,7.7500,NaN,Q
36,37,1,3,"Mamee, Mr. Hanna",male,NaN,0,0,2677,7.2292,NaN,C
42,43,0,3,"Kraeff, Mr. Theodor",male,NaN,0,0,349253,7.8958,NaN,C


In [9]:
# Save passengerIds for later
paxIds = df_missing_age['PassengerId'].tolist()
paxIds

[6, 18, 20, 27, 29, 30, 32, 33, 37, 43]

In [10]:
impute_age = df.pivot_table(values=["Age"], index=["Sex","Pclass"], aggfunc=np.mean)
print impute_age

                     Age
Sex    Pclass           
female 1       34.611765
       2       28.722973
       3       21.750000
male   1       41.281386
       2       30.740707
       3       26.507589


In [11]:
# Iterate only through rows with missing age
for i,row in df.loc[df['Age'].isnull(),:].iterrows():
  idx = tuple([row['Sex'],row['Pclass']])
  df.loc[i,'Age'] = impute_age.loc[idx].values[0]

In [12]:
# Now check the missing values again to confirm they don't exist anymore
df[df['Age'].isnull()].head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [13]:
# Check the new age values
df[df['PassengerId'].isin(paxIds)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,26.507589,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,30.740707,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,21.750000,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,26.507589,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,21.750000,0,0,330959,7.8792,NaN,Q
29,30,0,3,"Todoroff, Mr. Lalio",male,26.507589,0,0,349216,7.8958,NaN,S
31,32,1,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,34.611765,1,0,PC 17569,146.5208,B78,C
32,33,1,3,"Glynn, Miss. Mary Agatha",female,21.750000,0,0,335677,7.7500,NaN,Q
36,37,1,3,"Mamee, Mr. Hanna",male,26.507589,0,0,2677,7.2292,NaN,C
42,43,0,3,"Kraeff, Mr. Theodor",male,26.507589,0,0,349253,7.8958,NaN,C
